<a href="https://colab.research.google.com/github/2303A51867/2303A51867.aiml/blob/main/A3_PART_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from abc import ABC, abstractmethod
import math
import random

# Base Game class
class Game(ABC):
    def __init__(self, initial_state):
        self.initial = initial_state

    @abstractmethod
    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        pass

    @abstractmethod
    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        pass

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    @abstractmethod
    def utility(self, state, player):
        """Return the value of this final state to player."""
        pass

    def get_current_player(self, state):
        """Determine the current player based on the state."""
        return 'X' if state.count('X') <= state.count('O') else 'O'

# TicTacToe game subclass
class TicTacToe(Game):
    def __init__(self, initial_state=None):
        if initial_state is None:
            initial_state = [' '] * 9
        super().__init__(initial_state)

    def actions(self, state):
        return [i for i, cell in enumerate(state) if cell == ' ']

    def result(self, state, move):
        new_state = state[:]
        new_state[move] = self.get_current_player(state)
        return new_state

    def is_terminal(self, state):
        return any(self.utility(state, player) for player in ['X', 'O']) or ' ' not in state

    def utility(self, state, player):
        # Utility function to evaluate the state
        winning_positions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
            (0, 4, 8), (2, 4, 6)              # Diagonals
        ]
        for a, b, c in winning_positions:
            if state[a] == state[b] == state[c] == player:
                return 1 if player == 'X' else -1
        return 0

# MiniMax Search Algorithm
def minimax_search(game, state):
    """Search game tree to determine the best move; return (value, move) pair."""
    player = game.get_current_player(state)

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

# Alpha-Beta Search Algorithm
def alphabeta_search(game, state):
    """Search game to determine the best action; use alpha-beta pruning."""
    player = game.get_current_player(state)

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)

# Function to play the game
def play_game(game, strategies, verbose=False):
    """Play a turn-taking game. ‘strategies‘ is a {player name: function} dict,
    where function(state, game) is used to get the player’s move."""
    state = game.initial
    while not game.is_terminal(state):
        player = game.get_current_player(state)
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print(f'Player {player} move: {move}')
            print(state)
    return state

# Strategy functions
def random_strategy(game, state):
    return random.choice(game.actions(state))

def minimax_strategy(game, state):
    """Use the MiniMax algorithm to determine the best move."""
    _, move = minimax_search(game, state)
    return move

def alphabeta_strategy(game, state):
    """Use Alpha-Beta pruning to determine the best move."""
    _, move = alphabeta_search(game, state)
    return move

# Example usage
if __name__ == "__main__":
    game = TicTacToe()
    strategies = {'X': alphabeta_strategy, 'O': random_strategy}
    final_state = play_game(game, strategies, verbose=True)
    print("Final state:", final_state)


Player X move: 0
['X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
Player O move: 8
['X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O']
Player X move: 2
['X', ' ', 'X', ' ', ' ', ' ', ' ', ' ', 'O']
Player O move: 5
['X', ' ', 'X', ' ', ' ', 'O', ' ', ' ', 'O']
Player X move: 1
['X', 'X', 'X', ' ', ' ', 'O', ' ', ' ', 'O']
Final state: ['X', 'X', 'X', ' ', ' ', 'O', ' ', ' ', 'O']
